In [1]:
from h2ogpte import H2OGPTE
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re
import pathlib
import spotipy  
import requests

# Without Query 

In [2]:
# Read to get User's favourite genre
user = pd.read_csv('../../data/user.csv')
genre = user[user['UserID'] == 'sibyl'].iloc[:, -3:].to_csv(index=False, header=False).strip()
print(genre)


rock,r&b,pop


In [3]:
# In case, id fails, use backup of top ranked song 
music_genres = {
    "country": "4H2TRR9FjnnIwxGnIt9stO",
    "electronic": "6s0yNmp4Hd32wGx40T6uL8",
    "folk": "0fjYN9BylnRXMA3or3QSld",
    "hip hop": "5ByAIlEEnxYdvpnezg7HTX",
    "jazz": "7pKWTcPfT9mg2iAhobFHpS",
    "k-pop": "5FLuHcyUiAlq2wCoqVuqUa",
    "metal": "55tGzbf7CA1fnnlTy8HgDX",
    "pop": "3TGRqZ0a2l1LRblBkJoaDx",
    "r&b": "6mz1fBdKATx6qP4oP1I65G",
    "rap": "59e7E2LMPZ2bhW5G6aCwX8",
    "rock": "5QTxFnGygVM4jFQiBovmRo",
    "soul": "2sCf9tz6LHByczuVT7rqIx"
}

backup = genre.split(',')
backup = [music_genres[genre] for genre in backup if genre in music_genres]
print(backup)

['5QTxFnGygVM4jFQiBovmRo', '6mz1fBdKATx6qP4oP1I65G', '3TGRqZ0a2l1LRblBkJoaDx']


# With Query

In [5]:
client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key='sk-yG5q6JxtLgMS1Twv0mwZKbIVmJw5pNKGL6uU57p5IfYYJUq7',
)

Please install the correct version of H2OGPTE with `pip install h2ogpte==1.4.10`.
You can enable strict version checking by passing strict_version_check=True.


In [6]:
# Create Collection
collection_id = client.create_collection(
    name="MusicRecco",
    description="Music Reccommender",
)

In [7]:
# Read Data
songs = pd.read_excel('../../data/songs_short.xlsx')
user = pd.read_csv('../../data/user.csv')
history = pd.read_csv('../../data/listening_history.csv')

In [8]:
# Convert Data to Readable Format
songs_txt = songs.to_csv(index=False)
user_txt = user.to_csv(index=False)
history_txt = history.to_csv(index=False)

In [9]:
# Upload
songs_data = client.upload('songs_short.xlsx', open('../../data/songs_short.xlsx', 'rb'))
user_data = client.upload('user.txt', user_txt.encode())
history_data = client.upload('listening_history.txt', history_txt.encode())

In [ ]:
# Ingest the uploaded data
client.ingest_uploads(collection_id, [songs_data])

Job(id='fb9a4b86-761b-4696-ae41-615fadc32b4a', name='Adding documents', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 4, 11, 4, 41, 34, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='85b9e68942f3496da4025ba8d4b10488', status='Indexing done.'), JobStatus(id='e4bbbd2d3e8743b2ba530a1a34d93a27', status='Collecting done.')], errors=[], last_update_date=datetime.datetime(2024, 4, 11, 4, 41, 55, tzinfo=TzInfo(UTC)), duration='21s', duration_seconds=21.0)

In [ ]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [ ]:
# Query to get suggestion for genre
recc_query = 'I like the genres ' + genre + ',recommend me 1 song from each of the genre I like with its Spotify ID. The Spotify ID should be a 22-character alphanumeric code.'
with client.connect(chat_session_id) as session:
    answer = session.query(
        message=recc_query,
        system_prompt='MUST return spotify_id at the end of each song.',
        rag_config={"rag_type": "rag"},
    ).content

    bot_response = answer
    
print(bot_response)

Sure, based on the information provided in the document, here are my recommendations for one song from each of the genres you mentioned (rock, r&b, and pop) along with their Spotify IDs:

* Rock: "Breaking the Law" by Judas Priest. Spotify ID: 3ODF56f90W3qxDqxHAUi3v
* R&B: "I Want You" by Marvin Gaye. Spotify ID: 0xZuxFQjLWZnRMOFlFDsF5
* Pop: "Call Me Maybe" by Carly Rae Jepsen. Spotify ID: Carly Rae Jepsen

Note that the Spotify ID for Carly Rae Jepsen is just her name, as the IDs provided in the document are for specific songs rather than artists.


In [ ]:
# Convert query to spotify id to get name and album picture
pattern = r'\w{22}'

spotify_ids = re.findall(pattern, bot_response)
print(spotify_ids)

# In case, id fails, use backup of top ranked song 
music_genres = {
    "country": "4H2TRR9FjnnIwxGnIt9stO",
    "electronic": "6s0yNmp4Hd32wGx40T6uL8",
    "folk": "0fjYN9BylnRXMA3or3QSld",
    "hip hop": "5ByAIlEEnxYdvpnezg7HTX",
    "jazz": "7pKWTcPfT9mg2iAhobFHpS",
    "k-pop": "5FLuHcyUiAlq2wCoqVuqUa",
    "metal": "55tGzbf7CA1fnnlTy8HgDX",
    "pop": "3TGRqZ0a2l1LRblBkJoaDx",
    "r&b": "6mz1fBdKATx6qP4oP1I65G",
    "rap": "59e7E2LMPZ2bhW5G6aCwX8",
    "rock": "5QTxFnGygVM4jFQiBovmRo",
    "soul": "2sCf9tz6LHByczuVT7rqIx"
}

backup = genre.split(',')
backup = [music_genres[genre] for genre in backup if genre in music_genres]
spotify_ids.extend(backup)
print(spotify_ids)

# Spotify API

In [11]:
spotify_ids = ['3ODF56f90W3qxDqxHAUi3v', '0xZuxFQjLWZnRMOFlFDsF5', '5QTxFnGygVM4jFQiBovmRo', '6mz1fBdKATx6qP4oP1I65G', '3TGRqZ0a2l1LRblBkJoaDx']

In [12]:
url = "https://accounts.spotify.com/api/token"
payload = {
    "grant_type": "client_credentials",
    "client_id": "31e6d0a4c512492685193069c8450433",
    "client_secret": "054e3779e1974c038b277f4f2b5460ae"
}
headers = {
    "Content-Type": "application/x-www-form-urlencoded"
}

response = requests.post(url, data=payload, headers=headers)

access_token = response.json()['access_token']


In [13]:
def get_spotify_track_info(track_id, access_token):    
    # Spotify API endpoint for retrieving track information
    endpoint = f"https://api.spotify.com/v1/tracks/{track_id}"
    
    # Set up the request headers with the access token
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    # Make the GET request to Spotify API
    response = requests.get(endpoint, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        track_info = response.json()
        
        # Extract the track name and image URL from the response
        track_name = track_info['name']
        image_url = track_info['album']['images'][0]['url']  # Get the first image (largest size)
        
        return track_name, image_url
    else:
        # If the request failed, print the error message
        print(f"Error: {response.status_code} - {response.text}")
        return None, None


In [14]:
track_info_list = []

for track_id in spotify_ids:
    track_name, image_url = get_spotify_track_info(track_id, access_token)
    if track_name and image_url:
        track_info_list.append({"track_name": track_name, "image_url": image_url})
        if len(track_info_list) >= 3:
            break  # Stop adding more tracks once the list reaches a length of 3

print(track_info_list)

for track_info in track_info_list:
    print(f"Track Name: {track_info['track_name']}")
    print(f"Image URL: {track_info['image_url']}")
    print()  # Add a newline for better readability between track information


[{'track_name': 'Strawberry Wine', 'image_url': 'https://i.scdn.co/image/ab67616d0000b273b2145bcb28fd6dbaa1e44a83'}, {'track_name': 'Step On', 'image_url': 'https://i.scdn.co/image/ab67616d0000b2732216645273a04d24d42505a0'}, {'track_name': "(Don't Fear) The Reaper", 'image_url': 'https://i.scdn.co/image/ab67616d0000b2733ac318439ae56ce048d7bf5d'}]
Track Name: Strawberry Wine
Image URL: https://i.scdn.co/image/ab67616d0000b273b2145bcb28fd6dbaa1e44a83

Track Name: Step On
Image URL: https://i.scdn.co/image/ab67616d0000b2732216645273a04d24d42505a0

Track Name: (Don't Fear) The Reaper
Image URL: https://i.scdn.co/image/ab67616d0000b2733ac318439ae56ce048d7bf5d



# Chatbot Page

In [38]:
#Initiate Chat
chat_session_id = client.create_chat_session(collection_id)

In [39]:
with client.connect(chat_session_id) as session:
    answer = session.query(
        message= "My favourite genre is Rap, Pop and KPop. Recommend me some songs.",
        system_prompt='Reccommend songs in the songs_short.xlsx document only, and provide spotify_id after each song',
        rag_config={"rag_type": "rag"},
    ).content
    
    reccommendations = answer

# Convert to reccommend data save

In [3]:
import re

# Provided text containing song titles
text = """
Here are five songs that you might find comforting when you're feeling down, based on the information provided in the document:

1. "Yesterday" by The Beatles (page 17, row 1, spotify_id: "6FRwDxXsvSasw0y2eDArsz")
2. "Imagine" by John Lennon (page 17, row 2, spotify_id: "1bp2IO61zbQrbWNmKKxg3f")
3. "Hallelujah" by Leonard Cohen (page 17, row 3, spotify_id: "3qiyyUfYe7CRYLucrPmulD")
4. "Someone Like You" by Adele (page 18, row 1, spotify_id: "5uvosCdMlFdTXhoazkTI5R")
5. "Fix You" by Coldplay (page 18, row 2, spotify_id: "6mHOcVtsHLMuesJkswc0GZ")
"""

# Regular expression pattern to extract song titles
pattern = r'"([^"]+)"'

# Find all matches of the pattern in the text
song_titles = re.findall(pattern, text)

print(song_titles)


['Yesterday', '6FRwDxXsvSasw0y2eDArsz', 'Imagine', '1bp2IO61zbQrbWNmKKxg3f', 'Hallelujah', '3qiyyUfYe7CRYLucrPmulD', 'Someone Like You', '5uvosCdMlFdTXhoazkTI5R', 'Fix You', '6mHOcVtsHLMuesJkswc0GZ']
Yesterday
6FRwDxXsvSasw0y2eDArsz
Imagine
1bp2IO61zbQrbWNmKKxg3f
Hallelujah
3qiyyUfYe7CRYLucrPmulD
Someone Like You
5uvosCdMlFdTXhoazkTI5R
Fix You
6mHOcVtsHLMuesJkswc0GZ


# Link extraction testing

In [ ]:
def id_extract(text):
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    regex_pattern =  r'\b[A-Za-z0-9]{22}\b'

    # Extracting Spotify IDs
    spotify_ids = []
    for string in lines:
        match = re.findall(regex_pattern, string)
        if match:
            spotify_ids.append(match)
    
    #output clickable links
    for idx, spotify_id in enumerate(spotify_ids, start=1):
        link = f"https://open.spotify.com/track/{spotify_id[0]}"
        print(f"{link}")

In [ ]:
def spot_extraction(msg):
    with client.connect(chat_session_id) as session:
        # Simple Question for Document Collection
            answer = session.query(
            message= msg,
            system_prompt ='Recommend not more than 5 music. MUST return spotify_id at the end of each song.',
            rag_config={
                "rag_type": "rag",
            },
            ).content
    return answer

In [ ]:
ans = spot_extraction("What are some music that i can listen to when im sad")
print(ans)

Here are five songs that you might find comforting when you're feeling down, based on the information provided in the document:

1. "Yesterday" by The Beatles (page 17, row 1, spotify\_id: "6FRwDxXsvSasw0y2eDArsz")
2. "Imagine" by John Lennon (page 17, row 2, spotify\_id: "1bp2IO61zbQrbWNmKKxg3f")
3. "Hallelujah" by Leonard Cohen (page 17, row 3, spotify\_id: "3qiyyUfYe7CRYLucrPmulD")
4. "Someone Like You" by Adele (page 18, row 1, spotify\_id: "5uvosCdMlFdTXhoazkTI5R")
5. "Fix You" by Coldplay (page 18, row 2, spotify\_id: "6mHOcVtsHLMuesJkswc0GZ")

Please note that these song recommendations are based solely on the data provided in the document and do not take into account personal preferences or musical tastes.


In [ ]:
ans1 = spot_extraction("What are some music that you recommend me")
print(ans1)

Based on the data provided, here are five songs from different artists that I recommend:

1. "Light My Fire" by The Doors (spotify\_id: 5uvosCdMlFdTXhoazkTI5R)
2. "The Logical Song - 2010 Remastered" by Supertramp (spotify\_id: 6mHOcVtsHLMuesJkswc0GZ)
3. "Barracuda" by Heart (spotify\_id: 4KfSdst7rW39C0sfhArdrz)
4. "Like a Stone" by Audioslave (spotify\_id: 3YuaBvuZqcwN3CEAyyoaei)
5. "Back In Black" by AC/DC (spotify\_id: 08mG3Y1vljYA6bvDt4Wqkj)

These songs are from well-known rock bands and have high popularity scores according to the data.


In [ ]:
ans2 = spot_extraction("music that i can dance to the most. Return the spotify id too")

In [ ]:
print(ans2)

Based on the "danceability" feature in the given dataset, here are five songs that have high danceability scores:

1. "Kinetic (The Crystal Method vs. Dada Life)" by The Crystal Method, with a danceability score of 0.616. Spotify ID: 6s0yNmp4Hd32wGx40T6uL8
2. "Galvanize" by The Chemical Brothers, with a danceability score of 0.745. Spotify ID: 4bz7uB4edifWKJXSDxwHcs
3. "God Is a DJ" by Faithless, with a danceability score of 0.606. Spotify ID: 1pUFYb9peWkK8m1WCKNRjp
4. "In Yer Face - Bicep Remix" by 808 State, with a danceability score of 0.616. Spotify ID: 6DxKG8EHEqkWdKrFQSvm32
5. "Teardrop" by Massive Attack, with a danceability score of 0.615. Spotify ID: 67Hna13dNDkZvBpTXRIaOJ

Note that these recommendations are based solely on the danceability metric and do not take into account other factors such as genre, tempo, or personal preference.


In [ ]:
def make_link(text):
    lines = text.split('\n')
    lines = [line.strip() for line in lines if line.strip()]
    regex_pattern =  r'\b[A-Za-z0-9]{22}\b'

    # Extracting Spotify IDs
    spotify_ids = []
    for string in lines:
        match = re.findall(regex_pattern, string)
        if match:
            spotify_ids.append(match)
    
    #output clickable links
    for idx, spotify_id in enumerate(spotify_ids, start=1):
        link = f"https://open.spotify.com/track/{spotify_id[0]}"
        print(f"{link}")


In [ ]:
make_link(ans)

https://open.spotify.com/track/6FRwDxXsvSasw0y2eDArsz
https://open.spotify.com/track/1bp2IO61zbQrbWNmKKxg3f
https://open.spotify.com/track/3qiyyUfYe7CRYLucrPmulD
https://open.spotify.com/track/5uvosCdMlFdTXhoazkTI5R
https://open.spotify.com/track/6mHOcVtsHLMuesJkswc0GZ


In [ ]:
import re

def extract_spotify_ids(strings):
    # Regular expression pattern to match Spotify IDs
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"

    # Extracting Spotify IDs
    spotify_ids = []
    for string in strings:
        match = re.search(pattern, string)
        if match:
            spotify_ids.append(match.group(1))

    return spotify_ids

# Example usage:
strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    "2. \"The Sound of Silence\" by Simon & Garfunkel (page 18, row 2) - Spotify ID: 5uvosCdMlFdTXhoazkTI5R",
    "3. \"Imagine\" by John Lennon (page 19, row 1) - Spotify ID: 6mHOcVtsHLMuesJkswc0GZ",
    "4. \"Hallelujah\" by Leonard Cohen (page 20, row 2) - Spotify ID: 0sgDEFq9oeC8ueOK0mR5IA",
    "5. \"Someone Like You\" by Adele (page 20, row 3) - Spotify ID: 4KfSdst7rW39C0sfhArdrz",
    "6. \"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "7. \"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: mHOcVtsHLMuesJkswc0GZ)",
    "8. \"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "9. \"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "10. \"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)"
]

spotify_ids = extract_spotify_ids(strings)
print(spotify_ids)


['6FRwDxXsvSasw0y2eDArsz', '5uvosCdMlFdTXhoazkTI5R', '6mHOcVtsHLMuesJkswc0GZ', '0sgDEFq9oeC8ueOK0mR5IA', '4KfSdst7rW39C0sfhArdrz', '5uvosCdMlFdTXhoazkTI5R', '4KfSdst7rW39C0sfhArdrz', '3YuaBvuZqcwN3CEAyyoaei', '08mG3Y1vljYA6bvDt4Wqkj']


In [ ]:
import re

# Sample longer strings
longer_strings = [
    "1. \"Yesterday\" by The Beatles (page 17, row 1) - Spotify ID: 6FRwDxXsvSasw0y2eDArsz",
    'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
    'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'
]

# Regex pattern
regex_pattern = r'\b[A-Za-z0-9]{22}\b'

# Extract alphanumeric strings of length 22 from longer strings
for string in longer_strings:
    matches = re.findall(regex_pattern, string)
    for match in matches:
        print(match)


6FRwDxXsvSasw0y2eDArsz
6FRwDxXsvSasw0y2eDArsz
4KfSdst7rW39C0sfhArdrz


<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
<>:6: SyntaxWarning: invalid escape sequence '\_'
<>:7: SyntaxWarning: invalid escape sequence '\_'
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_84841/4172587299.py:6: SyntaxWarning: invalid escape sequence '\_'
  'spotify\_id: "6FRwDxXsvSasw0y2eDArsz")',
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_84841/4172587299.py:7: SyntaxWarning: invalid escape sequence '\_'
  'rt (spotify\_id: 4KfSdst7rW39C0sfhArdrz)'


In [ ]:
import re

def extract_spotify_id(text):
    pattern = r"(?:Spotify ID|spotify_id)[:\s]*\"?([a-zA-Z0-9_-]{22})\"?"
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    else:
        return None


# Test the function with your examples
strings = [
    "\"Yesterday\" by The Beatles (page 17, row 1, spotify_id: \"6FRwDxXsvSasw0y2eDArsz\")",
    "\"Imagine\" by John Lennon (page 17, row 2, spotify_id: \"1bp2IO61zbQrbWNmKKxg3f\")",
    "\"Light My Fire\" by The Doors (spotify_id: 5uvosCdMlFdTXhoazkTI5R)",
    "\"The Logical Song - 2010 Remastered\" by Supertramp (spotify_id: 6mHOcVtsHLMuesJkswc0GZ)",
    "\"Barracuda\" by Heart (spotify_id: 4KfSdst7rW39C0sfhArdrz)",
    "\"Like a Stone\" by Audioslave (spotify_id: 3YuaBvuZqcwN3CEAyyoaei)",
    "\"Back In Black\" by AC/DC (spotify_id: 08mG3Y1vljYA6bvDt4Wqkj)",
    "\"Kinetic (The Crystal Method vs. Dada Life)\" by The Crystal Method, with a danceability score of 0.616. Spotify ID: 6s0yNmp4Hd32wGx40T6uL8",
    "\"Galvanize\" by The Chemical Brothers, with a danceability score of 0.745. Spotify ID: 4bz7uB4edifWKJXSDxwHcs",
    "\"God Is a DJ\" by Faithless, with a danceability score of 0.606. Spotify ID: 1pUFYb9peWkK8m1WCKNRjp",
    "\"In Yer Face - Bicep Remix\" by 808 State, with a danceability score of 0.616. Spotify ID: 6DxKG8EHEqkWdKrFQSvm32",
    "\"Teardrop\" by Massive Attack, with a danceability score of 0.615. Spotify ID: 67Hna13dNDkZvBpTXRIaOJ",
    "\"Hallelujah\" by Leonard Cohen (page 17, row 3, spotify_id: \"3qiyyUfYe7CRYLucrPmulD\")",
    "\"Someone Like You\" by Adele (page 18, row 1, spotify_id: \"5uvosCdMlFdTXhoazkTI5R\")",
    "\"Fix You\" by Coldplay (page 18, row 2, spotify_id: \"6mHOcVtsHLMuesJkswc0GZ\")"
]

for string in strings:
    print(extract_spotify_id(string))


6FRwDxXsvSasw0y2eDArsz
1bp2IO61zbQrbWNmKKxg3f
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ
4KfSdst7rW39C0sfhArdrz
3YuaBvuZqcwN3CEAyyoaei
08mG3Y1vljYA6bvDt4Wqkj
6s0yNmp4Hd32wGx40T6uL8
4bz7uB4edifWKJXSDxwHcs
1pUFYb9peWkK8m1WCKNRjp
6DxKG8EHEqkWdKrFQSvm32
67Hna13dNDkZvBpTXRIaOJ
3qiyyUfYe7CRYLucrPmulD
5uvosCdMlFdTXhoazkTI5R
6mHOcVtsHLMuesJkswc0GZ


ignore below

In [ ]:
len("6DxKG8EHEqkWdKrFQSvm32")

22

In [ ]:
#convert to list of string



In [ ]:
# Function to convert Spotify IDs to clickable links
def spotify_id_to_link(spotify_id):
    return f"https://open.spotify.com/track/{spotify_id}"

# Output the clickable links
for idx, spotify_id in enumerate(spotify_ids, start=1):
    link = spotify_id_to_link(spotify_id)
    print(f"{idx}. {link}")


1. https://open.spotify.com/track/s
2. https://open.spotify.com/track/p
3. https://open.spotify.com/track/o
4. https://open.spotify.com/track/t
5. https://open.spotify.com/track/i
6. https://open.spotify.com/track/f
7. https://open.spotify.com/track/y
8. https://open.spotify.com/track/_
9. https://open.spotify.com/track/i
10. https://open.spotify.com/track/d
11. https://open.spotify.com/track/s
